In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [87]:
col_labels = ['xmeas%02d_low' % x for x in [1,2,3,8,9,21]]+ ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]]
#df_mat = [[0,0,0,0,1,0,0,0,0,0,0,1], np.zeros(12), np.zeros(12), np.zeros() 
df_mat = np.zeros([12,12], dtype=int)
ref_df = pd.DataFrame(df_mat, columns = col_labels, index=col_labels)
ref_df.loc['xmeas01_low'][['xmeas21_high', 'xmeas09_low']] = [1,1]
ref_df.loc['xmeas09_low']['xmeas21_high'] = 1

ref_df.loc['xmeas01_high'][['xmeas21_low', 'xmeas09_high']] = [1,1]
ref_df.loc['xmeas09_high']['xmeas21_low'] = 1

In [88]:
def generate_conf_df(df, df_ref):
    confMatrix = np.zeros([2,2], dtype=int)
     
    for col, col_ref in zip(df.columns, df_ref.columns):
        for row, row_ref in zip(df[col], df_ref[col_ref]):
            print(row, row_ref)
            if row == row_ref and row == 0:
                confMatrix[1,1] += 1
            elif row == row_ref and row == 1:
                confMatrix[0,0] += 1
            elif row != row_ref and row == 1:
                confMatrix[0,1] += 1
            else:
                confMatrix[1,0]+= 1 
    return pd.DataFrame(confMatrix,columns=["p", "n"], index = ["Y", "N"])

In [99]:
#roc fp*tp

def fp_rate(conf_df):
    return conf_df["n"]["Y"]/(conf_df["n"]["Y"]+ conf_df["n"]["N"])

def tp_rate(conf_df): #since it is positive what are the odds of being classsified as positive
    return conf_df['p']['Y']/(conf['p']['Y'] + conf['p']['N'])

def precision(conf_df): #from those i choose as correct, how many really were
    return conf_df['p']['Y']/(conf_df['p']['Y'] + conf_df['n']['Y'])
